In [1]:
def print_average_parameter(model):
    total_params = 0
    total_sum = 0.0
    for param in model.parameters():
        total_params += param.numel()
        total_sum += param.sum().item()
    average = total_sum / total_params
    print("Average parameter value:", average)

In [2]:
from models import mae_vit_base_patch16_test, mae_vit_large_patch16,Transconv_small_patch16_test
from transconv import TransConvRaw
from mae.models_vit import VisionTransformer as MAE_VisionTransformer
import torch
import os
import torch.nn as nn
from functools import partial
import copy
from nets.mae.models_mae import mae_vit_base_patch16, mae_vit_large_patch16
pretrain_vit_path = 'D:/Mengyu_Li/saved_models/mae_pretrain/mae_vit_base_patch16/model_latest.pt'
pre_trained_vit = mae_vit_base_patch16()
print_average_parameter(pre_trained_vit)
pre_trained_vit.load_state_dict(torch.load(pretrain_vit_path)['model_state_dict'])
print_average_parameter(pre_trained_vit)
tasks = ['extraction']
# image_vit = mae_vit_base_patch16_test(pretrained=pretrain_path)
# image_encoder.load_state_dict(torch.load(pretrain_path)['model_state_dict'])
# patch_size=16, channel_ratio=4, embed_dim=768 if pre_trained_vit is not None else 384, depth=12,
#                         num_heads=6, mlp_ratio=4, qkv_bias=True,
#                         pre_trained_vit=pre_trained_vit, vit_depth=12, pre_trained_conformer=pre_trained_conformer,
#                         additive_fusion_down=False, additive_fusion_up=False, up_ftr_map_size=[56]+[56]*3+[28]*4+[14]*4, down_ftr_map_size=[197]*12,
                        
# patch_size=16, embed_dim=768, depth=12, num_heads=12,
#         decoder_embed_dim=512, decoder_depth=8, decoder_num_heads=16,
#         mlp_ratio=4, norm_layer=partial(nn.LayerNorm, eps=1e-6),
        
model = TransConvRaw( patch_size=16, in_chans=3, base_channel=64, channel_ratio=4, num_med_block=0,
                 embed_dim=768, depth=12, num_heads=12, mlp_ratio=4., qkv_bias=True,
                 pre_trained_vit=pre_trained_vit, finetune_vit=True, vit_depth=12, pre_trained_conformer=None, finetune_conv=True,
                 additive_fusion_down=False, additive_fusion_up=False, up_ftr_map_size=[56]+[56]*3+[28]*4+[14]*4, down_ftr_map_size=[197]*12,
                 qk_scale=None, drop_rate=0., attn_drop_rate=0., drop_path_rate=0., decoder_embed_dim=512, out_chans=7)
# model_tran = MAE_VisionTransformer(
#         patch_size=16, embed_dim=768, depth=12, num_heads=12, mlp_ratio=4, qkv_bias=True,
#         norm_layer=partial(nn.LayerNorm, eps=1e-6), num_classes=1000, drop_path_rate=0.1,
#         global_pool=True)
# model_tran.load_state_dict(torch.load(pretrain_vit_path)['model_state_dict'], strict=False)

Average parameter value: 0.0012862317514475254
Average parameter value: 0.0008913620735884696


In [3]:
[56]+[56]*3+[28]*4+[14]*4

[56, 56, 56, 56, 28, 28, 28, 28, 14, 14, 14, 14]

In [4]:
pre_trained_vit.blocks

ModuleList(
  (0-11): 12 x Block(
    (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
    (attn): Attention(
      (qkv): Linear(in_features=768, out_features=2304, bias=True)
      (attn_drop): Dropout(p=0.0, inplace=False)
      (proj): Linear(in_features=768, out_features=768, bias=True)
      (proj_drop): Dropout(p=0.0, inplace=False)
    )
    (drop_path): Identity()
    (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
    (mlp): Mlp(
      (fc1): Linear(in_features=768, out_features=3072, bias=True)
      (act): GELU(approximate='none')
      (fc2): Linear(in_features=3072, out_features=768, bias=True)
      (drop): Dropout(p=0.0, inplace=False)
    )
  )
)

In [5]:
print_average_parameter(pre_trained_vit)

Average parameter value: 0.0008913620735884696


In [6]:
print_average_parameter(model.pre_trained_vit)

Average parameter value: 0.0008913620735884696


In [7]:
import torch
asd = torch.rand(8, 3, 224, 224)

In [8]:
asd_out = model(asd)

x_t before decoder pred  torch.Size([8, 197, 512])


In [9]:
for i in range(9,13):
    print(i)

9
10
11
12


In [10]:
from help_func import print_var_detail
asd_3 = torch.rand(8, 196, 224)

x_masked, mask, ids_restore = pre_trained_vit.random_masking(asd_3, mask_ratio=0)
pool = nn.AdaptiveAvgPool2d(1)
asd_pool = pool(torch.rand(8, 1024, 7, 7))
print(asd_pool.shape)

torch.Size([8, 1024, 1, 1])


In [11]:
print_var_detail(mask)

 is a  <class 'torch.Tensor'> with shape torch.Size([8, 196]) max:  tensor(0.) min:  tensor(0.)


In [12]:
print(asd_out[0].shape)

torch.Size([8, 7, 224, 224])


In [13]:
print(asd_out[1].shape)

torch.Size([8, 7, 224, 224])


In [14]:
stack = asd_out[2]
for x in stack[:-1]:
    print_var_detail(x)

In [15]:
# for i in range(4):
#     downsample_layer = stack.pop()
#     print_var_detail(downsample_layer)

In [16]:
# depth = 12
# step = depth // 3
# 
# stack_new = stack[:2]
# 
# for i in range(2,len(stack)-1,step):
#     print(i)
#     end = min(i + step, len(stack)-1)
#     stacked_tensor = torch.stack(stack[i:end])
#     stack_new.append(torch.mean(stacked_tensor, dim=0))


In [17]:
# ====== Construct model ======
from modeling.unet import Unet
_model = Unet(in_chans=3,out_chans=7, chans=64, if_classify= False,  dim=224, mlp_ratio = 32)


In [18]:
asd_out_unet = _model(asd)

down:  torch.Size([8, 64, 224, 224])
down:  torch.Size([8, 128, 112, 112])
down:  torch.Size([8, 256, 56, 56])
down:  torch.Size([8, 512, 28, 28])
output:  torch.Size([8, 1024, 14, 14])
up:  torch.Size([8, 512, 28, 28])
output after cat:  torch.Size([8, 1024, 28, 28])
up:  torch.Size([8, 256, 56, 56])
output after cat:  torch.Size([8, 512, 56, 56])
up:  torch.Size([8, 128, 112, 112])
output after cat:  torch.Size([8, 256, 112, 112])
up:  torch.Size([8, 64, 224, 224])
output after cat:  torch.Size([8, 128, 224, 224])
